In [2]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Path to your saved layout
TLOF_FILE_PATH = "tlof_output.json"

def prompt_user():
    return input("Enter your request (e.g., 'Create a TLOF with rotation 50'):\n")

def call_model_with_prompt(prompt, functions, func_name):
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
        messages=[{"role": "user", "content": prompt}],
        functions=functions,
        function_call={"name": func_name}
    )
    return json.loads(response.choices[0].message.function_call.arguments)

def generate_new_layout(user_prompt):
    # Use your full function definition here
    args = call_model_with_prompt(user_prompt, function_def, "generate_landing_surface_layout")

    # Apply safety limits here if needed
    print("✅ New TLOF layout created with user-specified fields.")
    print(json.dumps(args, indent=2))
    with open(TLOF_FILE_PATH, "w") as f:
        json.dump(args, f, indent=2)

def update_existing_layout(user_prompt):
    with open(TLOF_FILE_PATH, "r") as f:
        existing_json = json.load(f)

    update_args = call_model_with_prompt(
        user_prompt,
        [{
            "name": "update_fields",
            "description": "Extract only the fields being updated.",
            "parameters": {
                "type": "object",
                "properties": {
                    "rotation": {"type": "number"},
                    "markerColor": {"type": "string"},
                    # Add more updateable fields here
                }
            }
        }],
        "update_fields"
    )

    for key, value in update_args.items():
        if key in existing_json["TLOF"][0]["dimensions"]:
            existing_json["TLOF"][0]["dimensions"][key] = value
            print(f"✅ Updated '{key}' to {value}")
        else:
            print(f"⚠️ Field '{key}' not found in layout.")

    with open(TLOF_FILE_PATH, "w") as f:
        json.dump(existing_json, f, indent=2)
    print("📝 Updated layout saved.")

def main():
    user_prompt = prompt_user()
    if os.path.exists(TLOF_FILE_PATH):
        update_existing_layout(user_prompt)
    else:
        generate_new_layout(user_prompt)

main()


Enter your request (e.g., 'Create a TLOF with rotation 50'):
 Create a TLOF with rotation 50


NameError: name 'function_def' is not defined